## Dynamic Time Warping with PySpark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [17]:
!pip3 install fastdtw

     |████████████████████████████████| 143kB 31.3MB/s eta 0:00:01
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp36-cp36m-linux_x86_64.whl size=392093 sha256=9265e33ac04159c60b91af075563e50627e45bb33cd39265aab5f0fd159a5902
  Stored in directory: /home/cdsw/.cache/pip/wheels/7e/95/df/935f19af51d6ab63d416c103c0d6eae7048a8c2ecb00a1497f
Successfully built fastdtw
You should consider upgrading via the 'pip install --upgrade pip' command.


In [23]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import fastdtw

In [2]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-1")\
    .config("spark.yarn.access.hadoopFileSystems","s3a://demo-aws-2/")\
    .getOrCreate()

In [27]:
sc.stop()

In [28]:
conf = SparkConf().setAppName("spark_dtw")
sc   = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [29]:
data   = sc.textFile("household_power_consumption.txt")
header = data.first().split(';')
data   = data.map(lambda x: x.split(';')).filter(lambda x: x!=header)

In [30]:
df = data.toDF(header)


Bad key "backend.qt4" on line 43 in
/etc/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [31]:
# Use schema
df1 = df.withColumn('Sub_metering_1',df.Sub_metering_1.cast('float')) \
    .withColumn('Sub_metering_2',df.Sub_metering_2.cast('float')) \
    .withColumn('Sub_metering_3',df.Sub_metering_3.cast('float')) \
    .withColumn('Voltage',df.Voltage.cast('float'))

In [32]:
df1.show(5)

+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|      Date|    Time|Global_active_power|Global_reactive_power|Voltage|Global_intensity|Sub_metering_1|Sub_metering_2|Sub_metering_3|
+----------+--------+-------------------+---------------------+-------+----------------+--------------+--------------+--------------+
|16/12/2006|17:24:00|              4.216|                0.418| 234.84|          18.400|           0.0|           1.0|          17.0|
|16/12/2006|17:25:00|              5.360|                0.436| 233.63|          23.000|           0.0|           1.0|          16.0|
|16/12/2006|17:26:00|              5.374|                0.498| 233.29|          23.000|           0.0|           2.0|          17.0|
|16/12/2006|17:27:00|              5.388|                0.502| 233.74|          23.000|           0.0|           1.0|          17.0|
|16/12/2006|17:28:00|              3.666|                0.528

In [33]:
df2 = df1.groupby("Date").agg(F.collect_list("Sub_metering_1").alias('meter1'),F.collect_list("Sub_metering_2").alias('meter2'),F.collect_list("Sub_metering_3").alias('meter3'))

In [34]:
df2.show(5)

+---------+--------------------+--------------------+--------------------+
|     Date|              meter1|              meter2|              meter3|
+---------+--------------------+--------------------+--------------------+
|12/1/2008|[1.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|
|24/9/2008|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|[1.0, 1.0, 0.0, 1...|
|26/2/2008|[0.0, 0.0, 0.0, 0...|[1.0, 1.0, 1.0, 0...|[0.0, 0.0, 0.0, 0...|
|16/7/2010|[0.0, 0.0, 0.0, 0...|[0.0, 0.0, 0.0, 0...|[1.0, 1.0, 0.0, 1...|
|26/1/2010|[1.0, 1.0, 1.0, 1...|[0.0, 0.0, 0.0, 0...|[19.0, 19.0, 18.0...|
+---------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [35]:
def get_udf_distance(array1, array2):
    distance, path = fastdtw(array1, array2, dist=euclidean)
    return distance

In [36]:
udf_dtw = udf(get_udf_distance , FloatType())

In [37]:
df3 = df2.select('Date', udf_dtw(df2.meter1, df2.meter2).alias('dtw_distance (meter1-meter2)'))

In [38]:
df3.show()

Py4JJavaError: An error occurred while calling o395.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 15, 10.0.72.48, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/etc/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/etc/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/etc/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/etc/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/etc/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 334, in _batched
    for item in iterator:
  File "<string>", line 1, in <lambda>

In [1]:
import pandas as pd


Bad key "backend.qt4" on line 43 in
/etc/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
data = {
  "recency": "20",
  "history": "100",
  "used_discount": "0",
  "used_bogo": "1",
  "is_referral": "1",
  "channel": "2",
  "offer": "1"
}

In [7]:
df = pd.DataFrame(data, index=[0])
df.columns = ['recency', 'history', 'used_discount', 'used_bogo', 
                'is_referral', 'channel', 'offer']